<a href="https://colab.research.google.com/github/al-dayviid21/Create-and-Tune-CNN-Model/blob/main/Create_and_Optimize_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creating a CNN model and Optimizing using Keras Tuner

In [3]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.0 MB/s eta 0:00:00


In [4]:
# import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import hyperparameters

In [5]:
# load dataset
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
# scale data
train_images = train_images / 255.0
test_images = test_images / 255.0

In [7]:
# reshape data
train_images = train_images.reshape(train_images.shape + (1,))
test_images = test_images.reshape(test_images.shape +  (1,))

In [8]:
# create build model function
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=32),
          kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'      
          ),
      keras.layers.Dense(
          units=10,
          activation='softmax'
      )
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3])),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )
  return model

In [10]:
# instantiate random search
tuner_search = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    directory='output',
    project_name = 'Fashion Mnist'
)

In [11]:
# start search for best parameter
tuner_search.search(
    train_images, train_labels, epochs=3, validation_split=0.1
)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.10266666859388351

Best val_accuracy So Far: 0.9160000085830688
Total elapsed time: 00h 02m 59s


In [12]:
tuner_search.results_summary()

Results summary
Results in output/Fashion Mnist
Showing 10 best trials
Trial summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 3
dense_1_units: 128
learning_rate: 0.001
Score: 0.9160000085830688
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 5
conv_2_filter: 96
conv_2_kernel: 3
dense_1_units: 64
learning_rate: 0.001
Score: 0.9089999794960022
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 3
dense_1_units: 112
learning_rate: 0.01
Score: 0.8799999952316284
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
conv_2_filter: 96
conv_2_kernel: 5
dense_1_units: 128
learning_rate: 0.01
Score: 0.8583333492279053
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 5
conv_2_filter: 32
conv_2_kernel: 5
dense_1_units: 96
learning_rate: 0.01
Score: 0.10266666859388351


In [13]:
# get the best model
best_model = tuner_search.get_best_models(num_models=1)[0]

In [14]:
# build best model 
best_model.build(train_images.shape)

In [15]:
# print summary of model
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        73792     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 128)               4718720   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 4,795,082
Trainable params: 4,795,082
Non-trainable params: 0
_________________________________________________________________
